# ChurnSense: Business Framing & Descriptive Analysis

## Project Context
Customer churn represents a major source of revenue loss for subscription-based businesses. 
While predicting churn risk is useful, the real business challenge is deciding **which customers to intervene on** given limited retention budgets.

This notebook focuses on:
- clearly defining the business problem
- understanding what "churn" means in this dataset
- validating the target variable
- performing descriptive analysis to build intuition
- identifying data quality, bias, and leakage risks

No modeling decisions are finalized in this notebook.

## Business Problem

The core business question is **not**:

> "Can we predict churn accurately?"

Instead, the decision-oriented question is:

> **Given limited retention budget, which customers should we target to maximize expected retained value under uncertainty?**

This reframes churn modeling as a **decision-support problem**, not a pure prediction task.

Key constraints:
- Retention actions (discounts, incentives, outreach) cost money
- Intervening on all customers is infeasible
- Predictions are probabilistic, not certain

Therefore, the goal of ChurnSense is to:
1. Estimate churn risk **before churn occurs**
2. Rank customers by risk
3. Support intervention decisions using thresholds and cost–benefit tradeoffs

## Data Science Workflow Alignment

This project follows a standard, defensible data science workflow:

1. Business problem framing 
2. Data understanding & leakage assessment ← **this notebook**
3. Data preprocessing & feature handling
4. Baseline modeling
5. Evaluation using business-relevant metrics
6. Threshold tuning and intervention analysis
7. Deployment as a decision-support tool

This notebook focuses on **Steps 1–2**.

## Load Data

In [1]:
import pandas as pd
import numpy as np

DATA_PATH = "../data/raw/churn.csv"
df = pd.read_csv(DATA_PATH)

df.head()

,CustomerID,Gender,Age,Under30,SeniorCitizen,Married,Dependents,NumberofDependents,Country,State,...,TotalExtraDataCharges,TotalLongDistanceCharges,TotalRevenue,SatisfactionScore,CustomerStatus,ChurnLabel,ChurnScore,CLTV,ChurnCategory,ChurnReason
0,8779-QRDMV,Male,78,No,Yes,No,No,0,United States,California,...,20,0.00,59.65,3,Churned,Yes,91,5433,Competitor,Competitor offered more data
1,7495-OOKFY,Female,74,No,Yes,Yes,Yes,1,United States,California,...,0,390.80,1024.10,3,Churned,Yes,69,5302,Competitor,Competitor made better offer
2,1658-BYGOY,Male,71,No,Yes,No,Yes,3,United States,California,...,0,203.94,1910.88,2,Churned,Yes,81,3179,Competitor,Competitor made better offer
3,4598-XLKNJ,Female,78,No,Yes,Yes,Yes,1,United States,California,...,0,494.00,2995.07,2,Churned,Yes,88,5337,Dissatisfaction,Limited range of services
4,4846-WHAFZ,Female,80,No,Yes,Yes,Yes,1,United States,California,...,0,234.21,3102.36,2,Churned,Yes,67,2793,Price,Extra data charges


## Dataset Overview

We begin by examining:
- number of observations
- number of features
- column names and data types

This helps validate assumptions before any modeling.

In [2]:
df.shape

(7043, 50)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 50 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   CustomerID                     7043 non-null   object 
 1   Gender                         7043 non-null   object 
 2   Age                            7043 non-null   int64  
 3   Under30                        7043 non-null   object 
 4   SeniorCitizen                  7043 non-null   object 
 5   Married                        7043 non-null   object 
 6   Dependents                     7043 non-null   object 
 7   NumberofDependents             7043 non-null   int64  
 8   Country                        7043 non-null   object 
 9   State                          7043 non-null   object 
 10  City                           7043 non-null   object 
 11  ZipCode                        7043 non-null   int64  
 12  Latitude                       7043 non-null   f

In [4]:
df.columns.tolist()

['CustomerID',
 'Gender',
 'Age',
 'Under30',
 'SeniorCitizen',
 'Married',
 'Dependents',
 'NumberofDependents',
 'Country',
 'State',
 'City',
 'ZipCode',
 'Latitude',
 'Longitude',
 'Population',
 'Quarter',
 'ReferredaFriend',
 'Number_of_Referrals',
 'TenureinMonths',
 'Offer',
 'PhoneService',
 'AvgMonthlyLongDistanceCharges',
 'MultipleLines',
 'InternetService',
 'InternetType',
 'AvgMonthlyGBDownload',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtectionPlan',
 'PremiumTechSupport',
 'StreamingTV',
 'StreamingMovies',
 'StreamingMusic',
 'UnlimitedData',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharge',
 'TotalCharges',
 'TotalRefunds',
 'TotalExtraDataCharges',
 'TotalLongDistanceCharges',
 'TotalRevenue',
 'SatisfactionScore',
 'CustomerStatus',
 'ChurnLabel',
 'ChurnScore',
 'CLTV',
 'ChurnCategory',
 'ChurnReason']

## Target Variable: What Does "Churn" Mean Here?

Before modeling, we must clearly identify:
- which column represents churn
- how it is encoded
- whether it reflects **future information**

A valid churn target should represent whether a customer *eventually left*, 
not information that would only be known after churn occurred.

In [5]:
# Look at unique values of likely target columns
potential_targets = ["ChurnLabel", "Churn", "Exited", "is_churn"]

for col in potential_targets:
    if col in df.columns:
        print(col, df[col].value_counts(dropna=False))

ChurnLabel ChurnLabel
No     5174
Yes    1869
Name: count, dtype: int64


## Churn Base Rate

The churn base rate tells us:
- how common churn is
- how imbalanced the problem may be
- which evaluation metrics are appropriate

In churn problems, precision–recall metrics are often more informative than accuracy.

In [6]:
target_col = "ChurnLabel" if "ChurnLabel" in df.columns else "Churn"

df[target_col].value_counts(normalize=True)

ChurnLabel
No     0.73463
Yes    0.26537
Name: proportion, dtype: float64

## Descriptive Statistics: Numerical Features

We inspect central tendency and spread for numerical variables to:
- understand customer behavior
- identify potential outliers
- build intuition for churn drivers

In [7]:
df.describe()

,Age,NumberofDependents,ZipCode,Latitude,Longitude,Population,Number_of_Referrals,TenureinMonths,AvgMonthlyLongDistanceCharges,AvgMonthlyGBDownload,MonthlyCharge,TotalCharges,TotalRefunds,TotalExtraDataCharges,TotalLongDistanceCharges,TotalRevenue,SatisfactionScore,ChurnScore,CLTV
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,46.509726,0.468692,93486.070567,36.197455,-119.756684,22139.603294,1.951867,32.386767,22.958954,20.515405,64.761692,2280.381264,1.962182,6.860713,749.099262,3034.379056,3.244924,58.505040,4400.295755
std,16.750352,0.962802,1856.767505,2.468929,2.154425,21152.392837,3.001199,24.542061,15.448113,20.418940,30.090047,2266.220462,7.902614,25.104978,846.660055,2865.204542,1.201657,21.170031,1183.057152
min,19.000000,0.000000,90001.000000,32.555828,-124.301372,11.000000,0.000000,1.000000,0.000000,0.000000,18.250000,18.800000,0.000000,0.000000,0.000000,21.360000,1.000000,5.000000,2003.000000
25%,32.000000,0.000000,92101.000000,33.990646,-121.788090,2344.000000,0.000000,9.000000,9.210000,3.000000,35.500000,400.150000,0.000000,0.000000,70.545000,605.610000,3.000000,40.000000,3469.000000
50%,46.000000,0.000000,93518.000000,36.205465,-119.595293,17554.000000,0.000000,29.000000,22.890000,17.000000,70.350000,1394.550000,0.000000,0.000000,401.440000,2108.640000,3.000000,61.000000,4527.000000
75%,60.000000,0.000000,95329.000000,38.161321,-117.969795,36125.000000,3.000000,55.000000,36.395000,27.000000,89.850000,3786.600000,0.000000,0.000000,1191.100000,4801.145000,4.000000,75.500000,5380.500000
max,80.000000,9.000000,96150.000000,41.962127,-114.192901,105285.000000,11.000000,72.000000,49.990000,85.000000,118.750000,8684.800000,49.790000,150.000000,3564.720000,11979.340000,5.000000,96.000000,6500.000000


## Data Leakage Assessment

Data leakage occurs when features contain information that would not be available
at the time a prediction is made.

In churn datasets, common leakage columns include:
- churn reasons
- post-churn customer status
- vendor-generated churn scores

These variables may improve model performance artificially but invalidate business use.

In [8]:
leakage_candidates = [
    "CustomerStatus", 
    "ChurnScore", 
    "ChurnReason", 
    "ChurnCategory"
]

[c for c in leakage_candidates if c in df.columns]

['CustomerStatus', 'ChurnScore', 'ChurnReason', 'ChurnCategory']

## Note on Data Cleaning and Transformation

Data cleaning, encoding, and scaling are intentionally **deferred to the modeling pipeline** rather than performed directly in this notebook.

This ensures that:
- preprocessing is applied **consistently** during training and inference
- no data leakage is introduced through manual transformations
- the workflow remains reproducible and production-aligned

All feature preprocessing decisions are implemented using `scikit-learn` pipelines in subsequent modeling steps.

## Bias, Assumptions, and Limitations

Important considerations:
- This is observational data, not experimental data
- Causal conclusions cannot be drawn directly
- Customer behavior may be influenced by unobserved factors
- Historical churn patterns may not generalize perfectly

All modeling decisions in this project are made under uncertainty.
The goal is **better decisions**, not perfect prediction.

## Key Takeaways from Descriptive Analysis

- Approximately **26–27%** of customers in this dataset have churned, indicating a moderately imbalanced classification problem.
- Several columns contain **post-churn information** (e.g., churn reasons and customer status) and must be excluded to avoid data leakage.
- Numerical features exhibit wide ranges, reinforcing the need for **scaling in linear models**.
- This dataset is suitable for **risk ranking and decision support**, but not for causal inference.

These findings guide subsequent modeling, evaluation metrics, and threshold-based intervention decisions.